In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from my_functions import *

from scipy.optimize import curve_fit
from scipy.integrate import simpson

from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))

In [ ]:
## Load QSO catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Qso_jpas_mock_flam_train.cat')

my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12:-4] += 1
my_filter_order[28:-4] += 1
my_filter_order[43:-4] += 1

pm_flx = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
pm_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]
zspec = pd.read_csv(filename, sep=' ', usecols=[127]).to_numpy().reshape(-1, )

# Apply errors
np.random.seed(22)
# pm_flx += pm_err * np.random.normal(size=pm_err.shape)

plate = pd.read_csv(filename, sep=' ', usecols=[122]).to_numpy().reshape(-1, )
mjd = pd.read_csv(filename, sep=' ', usecols=[123]).to_numpy().reshape(-1, )
fiber = pd.read_csv(filename, sep=' ', usecols=[124]).to_numpy().reshape(-1, )

In [ ]:
Lya_fts = pd.read_csv('csv/Lya_fts.csv')
EW_lya = np.abs(Lya_fts.LyaEW)
w_lya = 1215.67 # A
N_sources = pm_flx.shape[1]
mag = flux_to_mag(pm_flx, w_central.reshape(-1, 1))
mag[np.isnan(mag)] = 99.
zspec_dist = cosmo.luminosity_distance(zspec).to(u.cm).value
L = EW_lya * Lya_fts.LyaCont * 1e-17 * 4*np.pi * zspec_dist**2
L = np.log10(L)
L[np.isnan(L)] = -99

In [ ]:
ew0min = 40
ewobsmin = 150

# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, ew0min)
lya_lines, lya_cont_lines = identify_lines(line, pm_flx, pm_err, first=True)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(
    pm_flx, pm_err, cont_est_other, cont_err_other, ewobsmin, obs=True
)
other_lines = identify_lines(line_other, pm_flx, pm_err)

line_lya_multiple = is_there_line(
    pm_flx, pm_err, cont_est_other, cont_err_lya, 0, obs=True
)

In [ ]:
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])
nice_lya = nice_lya_select(lya_lines, other_lines, pm_flx, pm_err, cont_est_other, z_Arr)
nice_lya = nice_lya & (mag[-2] < 24)

In [ ]:
np.random.seed(25)
selection = np.random.choice(np.where(nice_lya & (zspec > 2.5))[0], 15)
# for src in selection:
#    fig = plt.figure(figsize=(8, 6))
#    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])
#
#    ax.set_title('src{}'.format(src), fontsize=15)
#
#    plt.show()
#
#    print(plate[src], mjd[src], fiber[src])
#    print('z_spec = {}'.format(zspec[src]))
#    print('z_nb = {}'.format(z_Arr[src]))
#    print('Lya EW0 = {}'.format(EW_lya[src]))
#    print(src)

In [ ]:
# GAUSSIAN curve
def gauss(x, a, x0, sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

In [ ]:
tcurves = load_tcurves(load_filter_tags())

In [ ]:
from astropy.table import Table

In [ ]:
selec = selection[:10]

path = '/home/alberto/cosmos/SDSS_Spectra/downloadID/'
np.save(f'{path}plate.npy', plate[selec])
np.save(f'{path}mjd.npy', mjd[selec])
np.save(f'{path}fiber.npy', fiber[selec])

In [ ]:
spec_list = []
for pl, mj, fi in zip(plate[selec], mjd[selec], fiber[selec]):
    spec_name = f'spec-{pl}-{mj}-{fi:04d}.fits'
    spec_list.append(
        Table.read(f'/home/alberto/cosmos/SDSS_Spectra/fits/{spec_name}')
    )

In [ ]:
for i, src in enumerate(selec):
    lya_l = lya_lines[src]
    line_start = np.where(line[:, src])[0][0]

    spec = spec_list[i]

    rspec = NB_synthetic_photometry(spec['flux'] * 1e-17, 10 ** spec['loglam'],
        w_central[-3], nb_fwhm_Arr[-3])

    norm = pm_flx[-3, src] / rspec

    cont = (cont_est_other[lya_l, src] + cont_est_lya[lya_l, src]) * 0.5

    idx = line_start + 1
    line_stop = line_start
    while True:
        if line[idx, src]:
            line_stop = idx
        else: break
        idx += 1 ; continue

    nb_fit_range = slice(line_start, line_stop + 1)

    x_t_start = w_central[line_start] - nb_fwhm_Arr[line_start] * 0.5
    x_t_stop = w_central[line_stop] + nb_fwhm_Arr[line_stop] * 0.5

    s = (x_t_stop - x_t_start) / 3 * 0.5

    mu = (
        w_central[line_start] * pm_flx[line_start, src] ** 2 
        + w_central[line_stop] * pm_flx[line_stop, src] ** 2
    ) / (pm_flx[line_start, src] ** 2 + pm_flx[line_stop, src] ** 2)

    popt = ((2 * np.pi) ** -0.5 * s ** -1, mu, s)

    t_res = 10000 # Resolution for the t curve
    x_t = np.linspace(x_t_start, x_t_stop, t_res)
    composed_t = np.zeros(t_res)

    t_Arr = []
    for i in range(line_start, line_stop + 1):
        t_Arr.append(np.interp(x_t, tcurves['w'][i], tcurves['t'][i]))
        composed_t += t_Arr[-1]
    t_Arr = np.array(t_Arr)

    # Compute the area under the gaussian covolved with the filter T
    gaussian = gauss(x_t, *popt)
    gaussian[np.where(gaussian < cont)] = cont
    cvlvd_curve = gaussian * composed_t
    cvlvd_area = simpson(cvlvd_curve - cont, x_t)

    nb_area = simpson(
        np.sum(t_Arr * (pm_flx[nb_fit_range, src] - cont).reshape(-1,1), axis=0), x_t
    )

    correct_factor = nb_area / cvlvd_area

    fig = plt.figure(figsize=(14, 8))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src], set_ylim=False)

    ax.plot(10 ** spec['loglam'], spec['flux'] * 1e-17 * norm,
        c='black', alpha=0.4, zorder=-99)
    ax.plot(10 ** spec['loglam'], spec['model'] * 1e-17 * norm,
        c='red', zorder=-100)

    w_x = w_central[line_start - 2 : line_stop + 2]
    ax.plot(x_t, gaussian * correct_factor, linestyle='--', linewidth=3, c='black')

    ax.plot(mu, cont, marker='s', markersize=12, c='C6')

    ymaxlim = np.max(spec['model'] * 1e-17 * norm) * 1.3
    yminlim = 0 - 0.2 * ymaxlim
    ax.set_ylim((yminlim, ymaxlim))
    ax.set_xlim((3500, 6000))

    plt.savefig('output/gauss_fit-src' + str(src) + 's.pdf', bbox_inches='tight', dpi=600)
    plt.show()

    ## L computation
    profile = gaussian * correct_factor
    profile[np.where(profile < cont)] = cont
    src_dist = cosmo.luminosity_distance(z_Arr[src]).to(u.cm).value
    real_dist = cosmo.luminosity_distance(zspec[src]).to(u.cm).value
    L = simpson(profile - cont, x_t) * 2*np.pi * src_dist ** 2
    L_real = Lya_fts.LyaEW[src] * Lya_fts.LyaCont[src] * 2*np.pi * real_dist ** 2 / (1 + zspec[src]) * 1e-17
    print('My L = {0:0.2f}'.format(np.log10(L)))
    print('Real L = {0:0.2f}'.format(np.log10(L_real)))

In [ ]:
N_nice_lya = len(np.where(nice_lya)[0])
L_nb = np.zeros(N_nice_lya)
L_sp = np.copy(L_nb)
for i, src in enumerate(np.where(nice_lya)[0]):
    lya_l = lya_lines[src]
    line_start = np.where(line[:, src])[0][0]

    cont = (cont_est_other[lya_l, src] + cont_est_lya[lya_l, src]) * 0.5

    idx = line_start + 1
    line_stop = line_start
    while True:
        if line[idx, src]:
            line_stop = idx
        else: break
        idx += 1 ; continue

    nb_fit_range = slice(line_start, line_stop + 1)

    x_t_start = w_central[line_start] - nb_fwhm_Arr[line_start] * 0.5
    x_t_stop = w_central[line_stop] + nb_fwhm_Arr[line_stop] * 0.5

    s = (x_t_stop - x_t_start) / 3 * 0.5

    mu = (
        w_central[line_start] * pm_flx[line_start, src] ** 2 
        + w_central[line_stop] * pm_flx[line_stop, src] ** 2
    ) / (pm_flx[line_start, src] ** 2 + pm_flx[line_stop, src] ** 2)

    popt = ((2 * np.pi) ** -0.5 * s ** -1, mu, s)

    t_res = 10000 # Resolution for the t curve
    x_t = np.linspace(x_t_start, x_t_stop, t_res)
    composed_t = np.zeros(t_res)

    t_Arr = []
    for j in range(line_start, line_stop + 1):
        t_Arr.append(np.interp(x_t, tcurves['w'][j], tcurves['t'][j]))
        composed_t += t_Arr[-1]
    t_Arr = np.array(t_Arr)

    # Compute the area under the gaussian covolved with the filter T
    gaussian = gauss(x_t, *popt)
    #gaussian[np.where(gaussian < cont)] = cont
    cvlvd_curve = gaussian * composed_t
    cvlvd_area = simpson(cvlvd_curve - cont, x_t)

    nb_area = simpson(
        np.sum(t_Arr * (pm_flx[nb_fit_range, src] - cont).reshape(-1,1), axis=0), x_t
    )

    correct_factor = nb_area / cvlvd_area

    ## L computation
    profile = gaussian * correct_factor
    profile[np.where(profile < cont)] = cont
    src_dist = cosmo.luminosity_distance(z_Arr[src]).to(u.cm).value
    real_dist = cosmo.luminosity_distance(zspec[src]).to(u.cm).value
    L_nb[i] = np.log10(simpson(profile - cont, x_t) * 2*np.pi * src_dist ** 2)
    L_sp[i] = np.log10(np.abs(Lya_fts.LyaF[src]) * 2*np.pi * real_dist ** 2 * 1e-17)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

ax.scatter(L_sp, L_nb, color='black', s=2)
line_x = np.linspace(40, 48, 100)
ax.plot(line_x, line_x, linestyle='--', color='red', label='1:1')

ax.set_ylabel('Estimated log L', fontsize=15)
ax.set_xlabel('Spectroscopic log L', fontsize=15)
ax.legend(fontsize=15, loc=2)

ax.set_ylim((40, 48))
ax.set_xlim((40, 48))

plt.show()